# POS tagging

Authors: *Andrea Jaunarena*, *Markel Ferro*, *Juliana Planas* and *Lukas Arana*

The purpose of this Jupyter notebook is to show the results and error analysis of the model implemented in our_HMM.py.  

Two different types of experiments can be found in this notebook. 

* The implemented algorithm will be tested in an in-domain fashion. In other words, the same dataset, split in train and test, will be used to evaluate the resulting models.

* The trained models by the algorithm will be evaluated in test data for which they have not been trained in order to test the capacity of the models to generalize to new domains/

## In-domain experiments

### Imports

In [1]:
from our_HMM import *

ModuleNotFoundError: No module named 'conllu'

## Out-domain experiments